In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.7 MB/s eta 0:00:00


In [ ]:
# Loading dataset from huggingFace
from datasets import load_dataset
ag_news_dataset = load_dataset('sh0416/ag_news')

README.md:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/33.7M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
ag_news_dataset['train']['description'][4]

'AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace barely three months before the US presidential elections.'

In [ ]:
# Convering to parquret
ag_news_dataset['train'].to_parquet('train.parquet')

Creating parquet from Arrow format:   0%|          | 0/120 [00:00<?, ?ba/s]

30177303

In [ ]:
# spark_df =ag_news_dataset['train'].to_parquet('train.parquet')

Creating parquet from Arrow format:   0%|          | 0/120 [00:00<?, ?ba/s]

In [ ]:
spark_df

30177303

In [ ]:
!pip install pyspark datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 13.7 MB/s eta 0:00:00


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, lower, count
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import datetime

In [ ]:
# Getting curent datetime
# YYYYMMDD = datetime.datetime.now().strftime('%Y%m%d')
# YYYYMMDD

'20250307'

In [ ]:
#  spark session
spark = SparkSession.builder.appName('WordCount').getOrCreate()

In [ ]:
# loading paquret file
spark_df = spark.read.parquet("train.parquet")

In [ ]:
spark_df.show()

+-----+--------------------+--------------------+
|label|               title|         description|
+-----+--------------------+--------------------+
|    3|Wall St. Bears Cl...|Reuters - Short-s...|
|    3|Carlyle Looks Tow...|Reuters - Private...|
|    3|Oil and Economy C...|Reuters - Soaring...|
|    3|Iraq Halts Oil Ex...|Reuters - Authori...|
|    3|Oil prices soar t...|AFP - Tearaway wo...|
|    3|Stocks End Up, Bu...|Reuters - Stocks ...|
|    3|Money Funds Fell ...|AP - Assets of th...|
|    3|Fed minutes show ...|USATODAY.com - Re...|
|    3|Safety Net (Forbe...|Forbes.com - Afte...|
|    3|Wall St. Bears Cl...| NEW YORK (Reuter...|
|    3|Oil and Economy C...| NEW YORK (Reuter...|
|    3|No Need for OPEC ...| TEHRAN (Reuters)...|
|    3|Non-OPEC Nations ...| JAKARTA (Reuters...|
|    3|Google IPO Auctio...| WASHINGTON/NEW Y...|
|    3|Dollar Falls Broa...| NEW YORK (Reuter...|
|    3|Rescuing an Old S...|If you think you ...|
|    3|Kids Rule for Bac...|The purchasing po...|


In [ ]:
# splits the word using blank
# create a column named word
# filters the words column using president
#  get the final count
# spark_df.select(explode(split(col('description'), ' ')).alias('word')).filter(col('word') == 'president').count()

1244

In [ ]:
# # words to count
# words_to_count = ["president", "the", "Asia"]
# # defining function to count the words
# def word_count(word):
#         return spark_df.select(explode(split(col('description'), ' ')).alias('word')) \
#                  .filter(col('word') == word) \
#                  .count()

# result = [(word, word_count(word)) for word in words_to_count]

In [ ]:
# result

[('president', 1244), ('the', 172579), ('Asia', 290)]

In [ ]:
#  defining scehma for results
# schema = StructType([
#         StructField('word', StringType(), True),
#         StructField('word_count', IntegerType(), True)
#     ])

In [ ]:
#  creating a datframe for result with availbale schema
# result_df = spark.createDataFrame(result, schema)

In [ ]:
result_df.show()

+---------+----------+
|     word|word_count|
+---------+----------+
|president|      1244|
|      the|    172579|
|     Asia|       290|
+---------+----------+



In [ ]:
# writing the file in paquret format
# result_df.write.mode('overwrite').parquet(f'word_count_{YYYYMMDD}.parquet')

In [ ]:
# all in one function
def process_specific_words(df, words_to_count=None):

    YYYYMMDD = datetime.datetime.now().strftime('%Y%m%d')

    if words_to_count:
        # Count occurrences of specific words
        result = df.select(explode(split(col('description'), ' ')).alias('word')) \
                   .filter(col('word').isin(words_to_count)) \
                   .groupBy('word') \
                   .agg(count('*').alias('word_count'))
        name = "given_words"
    else:
        # Count occurrences of all words
        result = df.select(explode(split(col('description'), ' ')).alias('word')) \
                   .groupBy('word') \
                   .agg(count('*').alias('word_count'))
        name = "all_words"

    # Save as Parquet
    result.write.mode('overwrite').parquet(f'word_count_{YYYYMMDD}_{name}.parquet')

    return result

In [ ]:
words_to_count = ["president", "the", "Asia"]
rr = process_specific_words(spark_df,words_to_count)
rr.show()

+---------+----------+
|     word|word_count|
+---------+----------+
|      the|    172579|
|president|      1244|
|     Asia|       290|
+---------+----------+



In [ ]:
df_all = process_specific_words(spark_df)

In [ ]:
df_all.show()

+--------------+----------+
|          word|word_count|
+--------------+----------+
|        doubts|       106|
|          hope|       450|
|neighborhoods.|         5|
|           ...|      1686|
|       lights,|         9|
|         those|       799|
|         pack,|         4|
|            By|      1501|
|       office?|         2|
|          some|      2990|
|         still|      1589|
|      working,|         9|
|        waters|        98|
|   transmitted|        13|
|           few|       851|
|        Ansari|        64|
|     standards|       146|
|  commercials,|         6|
|   the\future.|         1|
|     involving|       220|
+--------------+----------+
only showing top 20 rows

